# 임직원 및 도서 정보를 토대로 한 도서 추천 시스템

In [5]:
# 데이터 로드 및 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

profile = pd.read_excel('4조_데이터.xlsx', sheet_name='가상인사자료(전처리 - 산식 제거)')
order = pd.read_excel('4조_데이터.xlsx', sheet_name='기간별부서별주문목록')
review = pd.read_excel('4조_데이터.xlsx', sheet_name='서평데이터')

In [ ]:
# Train/Test Split: 전체 데이터를 기준으로 80:20 비율로 분리
train, test = train_test_split(order, test_size=0.2, random_state=42)

print("Train/Test 데이터 분리 완료")
print(f"Train 데이터 크기: {train.shape}")
print(f"Test 데이터 크기: {test.shape}")

In [ ]:
# Item-Based 추천: "소분류" 활용해 특징 벡터화
okt = Okt()

# Okt로 텍스트 토큰화
def tokenize_korean(text):
    tokens = okt.morphs(text)
    return " ".join(tokens)

# Train 데이터에서 "소분류"를 활용
train['processed_features'] = train['소분류'].fillna("").apply(tokenize_korean)

# TF-IDF로 특징 벡터화
tfidf = TfidfVectorizer()
book_features_matrix = tfidf.fit_transform(train['processed_features'])

# 도서 간 코사인 유사도 계산
book_similarity = cosine_similarity(book_features_matrix)

# 유사도 데이터프레임 생성
book_similarity_df = pd.DataFrame(book_similarity, index=train['상품명'], columns=train['상품명'])

print("도서 간 유사도 계산 완료")
print(book_similarity_df.head())

In [ ]:
# Review 데이터에서 "만족도 평가"를 평균 점수로 계산
train_review = review.merge(train, left_on="ISBN", right_on="상품코드", how="inner")
train_review['만족도 평가'] = pd.to_numeric(train_review['만족도 평가'], errors="coerce").fillna(0)

# 도서별 평균 만족도 계산
book_ratings = train_review.groupby('도서명')['만족도 평가'].mean()
print("도서별 평균 만족도 계산 완료")
print(book_ratings.head())

In [ ]:
# Train 데이터에서 도서 판매량 집계
book_popularity = train.groupby('상품명')['수량'].sum()
print("도서별 판매량 집계 완료")
print(book_popularity.head())

# "베스트셀러 여부"를 가중치로 반영
bestseller_weights = train.set_index('상품명')['베스트셀러 여부'].apply(lambda x: 1.2 if x == "해당" else 1.0)
print("베스트셀러 여부 가중치 적용 완료")
print(bestseller_weights.head())

In [ ]:
# Item-Based 추천: 특정 도서와 유사한 도서 추천
book_name = '트렌드 코리아 2024'  # 추천 기준 도서

# 유사 도서 상위 5개 추출
similar_books = book_similarity_df[book_name]
similar_books = similar_books[similar_books.index != book_name]  # 자기 자신 제외
similar_books = similar_books.iloc[:, 0]
similar_books = similar_books.drop_duplicates()
similar_books = similar_books.sort_values(ascending=False)  # 유사도 기준 정렬
similar_books = similar_books.head(5)

print("추천된 유사 도서:")
print(similar_books)
